In [ ]:
from pyspark.sql import SparkSession

# 1. Criar a SparkSession com suporte ao Hive
# As configurações já estão no ambiente, então o código fica limpo!
spark = (
    SparkSession.builder
    .appName("HiveMetastore-Test")
    .master("spark://spark-master:7077")
    .enableHiveSupport()  # Habilita a integração com o Hive Metastore
    .getOrCreate()
)

print("Spark Session com Hive Metastore criada!")

# 2. Criar um DataFrame de exemplo
data = [("Fusca", 1970), ("Opala", 1980), ("Chevette", 1985)]
columns = ["modelo", "ano"]
df = spark.createDataFrame(data, columns)

# 3. Salvar o DataFrame como uma tabela gerenciada
# O Spark irá gerenciar os metadados (no PostgreSQL) e os dados (no MinIO)
table_name = "carros_classicos"
print(f"Salvando DataFrame como tabela gerenciada: '{table_name}'")

df.write.mode("overwrite").saveAsTable(table_name)

print("Tabela salva com sucesso!")

# 4. Ler a tabela de volta usando spark.read.table()
print(f"\nLendo a tabela '{table_name}' do catálogo:")
df_lido = spark.read.table(table_name)
df_lido.show()

# 5. Você pode listar as tabelas existentes
print("\nTabelas no catálogo:")
spark.catalog.listTables().show()

# Você pode fechar este notebook, reiniciar o kernel ou até mesmo os contêineres.
# Ao abrir um novo notebook e executar a célula 4 novamente, a tabela ainda estará lá!

spark.stop()


In [ ]:
from pyspark.sql import SparkSession

# 1. Inicie a SparkSession com suporte ao Hive
# Lembre-se que todas as configurações já estão no ambiente, então o código fica limpo.
spark = (
    SparkSession.builder
    .appName("Gerenciando-Databases")
    .master("spark://spark-master:7077")
    .enableHiveSupport()  # Essencial para interagir com o catálogo do Hive
    .getOrCreate()
)

print("Spark Session com Hive Metastore criada!")

# 2. Crie um novo database chamado 'bronze_layer'
# Usar 'IF NOT EXISTS' é uma boa prática para evitar erros se o database já existir.
db_name = "bronze_layer"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name} COMMENT 'Database para dados brutos'")

print(f"\nDatabase '{db_name}' criado com sucesso!")

# 3. Verifique se o database foi criado listando todos os databases
print("\nDatabases existentes no catálogo:")
spark.sql("SHOW DATABASES").show()

# 4. Mude o contexto para o novo database
spark.sql(f"USE {db_name}")
print(f"\nContexto alterado para o database '{db_name}'.")

# 5. Crie uma tabela dentro do novo database
# Os dados desta tabela serão armazenados em MinIO no caminho:
# s3a://database/warehouse/bronze_layer.db/clientes_brutos
data = [("1", "John Doe", "2024-10-26"), ("2", "Jane Smith", "2024-10-27")]
columns = ["id_cliente", "nome_completo", "data_cadastro"]
df_clientes = spark.createDataFrame(data, columns)

df_clientes.write.mode("overwrite").saveAsTable("clientes_brutos")

print("\nTabela 'clientes_brutos' criada dentro do database 'bronze_layer'.")

# 6. Liste as tabelas no database atual para confirmar
print(f"\nTabelas no database '{db_name}':")
spark.sql("SHOW TABLES").show()

# Para acessar a tabela, você pode usar o nome qualificado: bronze_layer.clientes_brutos
# ou, como já estamos usando o database, apenas o nome da tabela.
print("\nLendo dados da nova tabela:")
spark.read.table("clientes_brutos").show()

# Lembre-se de parar a sessão ao final
spark.stop()


In [1]:
from pyspark.sql import SparkSession

# 1. Inicie a SparkSession com suporte ao Hive
# Lembre-se que todas as configurações já estão no ambiente, então o código fica limpo.
spark = (
    SparkSession.builder
    .appName("Gerenciando-Databases2")
    .master("spark://spark-master:7077")
    .enableHiveSupport()  # Essencial para interagir com o catálogo do Hive
    .getOrCreate()
)

print("Spark Session com Hive Metastore criada!")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/24 20:03:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session com Hive Metastore criada!


In [ ]:

# 2. Crie um novo database chamado 'bronze_layer'
# Usar 'IF NOT EXISTS' é uma boa prática para evitar erros se o database já existir.
db_name = "bronze_layer"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name} COMMENT 'Database para dados brutos'")

print(f"\nDatabase '{db_name}' criado com sucesso!")


# 4. Mude o contexto para o novo database
spark.sql(f"USE {db_name}")
print(f"\nContexto alterado para o database '{db_name}'.")

# 5. Crie uma tabela dentro do novo database
# Os dados desta tabela serão armazenados em MinIO no caminho:
# s3a://database/warehouse/bronze_layer.db/clientes_brutos
input_path = f"s3a://raw/cad_fi_hist_admin.csv"
df_clientes = spark.read.csv(input_path, sep=";", header=True)
df_clientes.show()
# df_clientes.write.mode("overwrite").saveAsTable("clientes_brutos")

# print("\nTabela 'clientes_brutos' criada dentro do database 'bronze_layer'.")

# # 6. Liste as tabelas no database atual para confirmar
# print(f"\nTabelas no database '{db_name}':")
# spark.sql("SHOW TABLES").show()

# # Para acessar a tabela, você pode usar o nome qualificado: bronze_layer.clientes_brutos
# # ou, como já estamos usando o database, apenas o nome da tabela.
# print("\nLendo dados da nova tabela:")
# spark.read.table("clientes_brutos").show()

# # Lembre-se de parar a sessão ao final
# spark.stop()

In [ ]:
print(df_clientes.count())

In [8]:

# 2. Crie um novo database chamado 'bronze_layer'
# Usar 'IF NOT EXISTS' é uma boa prática para evitar erros se o database já existir.
db_name = "bronze_layer"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name} COMMENT 'Database para dados brutos'")

print(f"\nDatabase '{db_name}' criado com sucesso!")


# 4. Mude o contexto para o novo database
spark.sql(f"USE {db_name}")
print(f"\nContexto alterado para o database '{db_name}'.")

# 5. Crie uma tabela dentro do novo database
# Os dados desta tabela serão armazenados em MinIO no caminho:
# s3a://database/warehouse/bronze_layer.db/clientes_brutos
input_path = f"s3a://raw/3A_Superstore/Orders.csv"
df_clientes = spark.read.csv(input_path, header=True)
df_clientes.show()
df_clientes.write.mode("overwrite").saveAsTable("Orders")
# df_clientes.write.mode("overwrite").saveAsTable("clientes_brutos")

# print("\nTabela 'clientes_brutos' criada dentro do database 'bronze_layer'.")

# # 6. Liste as tabelas no database atual para confirmar
# print(f"\nTabelas no database '{db_name}':")
# spark.sql("SHOW TABLES").show()

# # Para acessar a tabela, você pode usar o nome qualificado: bronze_layer.clientes_brutos
# # ou, como já estamos usando o database, apenas o nome da tabela.
# print("\nLendo dados da nova tabela:")
# spark.read.table("clientes_brutos").show()

# # Lembre-se de parar a sessão ao final
# spark.stop()


Database 'bronze_layer' criado com sucesso!

Contexto alterado para o database 'bronze_layer'.
+--------+---------+-------------------+------+----------------+------------------+
| ORDERID|BRANCH_ID|              DATE_|USERID|     NAMESURNAME|       TOTALBASKET|
+--------+---------+-------------------+------+----------------+------------------+
| 7905270|  320-DE1|2022-08-22 00:00:00| 72946|       Ali İlhan|2637,5499999999997|
| 8131447|   56-AN4|2022-06-05 00:00:00| 58126|      Aysun Dinç|           2262,06|
|10176430|  348-MU1|2023-01-02 00:00:00| 41317|     Taner Yavuz|           2195,54|
| 8445704|   39-AY3|2021-01-28 00:00:00| 39303| Esra Lara Keleş|            446,86|
| 8616360|  777-YA1|2022-10-24 00:00:00| 64870|       Ela Çakır|            430,18|
| 7369024|  716-BU3|2021-05-21 00:00:00| 89153|     Selin İlhan|            996,81|
| 7656284|   56-AN1|2022-12-25 00:00:00| 55529|   Yelda Erdoğan|            784,26|
| 7468769|  146-KA3|2023-04-29 00:00:00| 23596|    Hatice Çoban|

25/10/24 20:30:07 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/10/24 20:30:07 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
25/10/24 20:30:07 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/24 20:30:07 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [9]:
print(df_clientes.count())

[Stage 19:==============>                                         (6 + 18) / 24]

10235193


In [10]:
input_path = f"s3a://raw/3A_Superstore/Order_Details.csv"
df_Order_Details = spark.read.csv(input_path, header=True)
df_Order_Details.show()
print(df_Order_Details.count())
df_Order_Details.write.mode("overwrite").saveAsTable("Order_Details")


+-------+-------------+------+---------+------------------+------+--------+
|ORDERID|ORDERDETAILID|AMOUNT|UNITPRICE|        TOTALPRICE|ITEMID|ITEMCODE|
+-------+-------------+------+---------+------------------+------+--------+
|5523363|     27622768|     4|     12,5|             25,96| 23667|   40199|
|5503074|     27521136|     3|      3,5|             10,08|  1324|    2871|
|5582877|     27921491|     4|      4,9|             35,04|  4735|   17605|
|5571722|     27865830|     8|     15,5|             85,84| 17179|   32797|
|5635480|     28184303|     2|     57,5|             121,1| 21009|   34535|
|5614687|     28079597|     4|     14,7|              54,4| 22870|   44617|
|5700834|     28511213|     8|       15|            123,36| 22075|   20934|
|5684943|     28431460|     8|     13,5|               122|  2434|    8106|
|5684943|     28431459|     4|      9,4|             41,84|  4737|   17608|
|5684943|     28431462|     4|     10,5|             47,52| 13263|   14368|
|5685078|   

51185032


In [5]:
from pyspark.sql.functions import input_file_name

# Defina o bucket e o prefixo (diretório) que você quer listar
bucket_name = "raw"
prefix_path = "3A_Superstore/" # Deixe vazio para a raiz do bucket

# Use um curinga (glob) para que o Spark encontre todos os arquivos
# O formato "text" é leve e bom para isso.
try:
    path_para_ler = f"s3a://{bucket_name}/{prefix_path}*"

    # O Spark lê os arquivos (apenas o suficiente para obter os metadados do caminho)
    df_files = spark.read.format("text").load(path_para_ler)

    # Use input_file_name() para obter o caminho completo e distinct() para evitar duplicatas
    df_file_list = df_files.select(input_file_name().alias("caminho_completo")).distinct()

    print(f"Arquivos encontrados em 's3a://{bucket_name}/{prefix_path}':")
    df_file_list.show(truncate=False)

    # Se você quiser a lista em formato Python:
    file_list = [row.caminho_completo for row in df_file_list.collect()]
    print("\nLista de Arquivos (Python):")
    for f in file_list:
        print(f)

except Exception as e:
    print(f"Erro ao listar arquivos: {e}")
    print("Verifique se o bucket e o caminho existem e se as credenciais estão corretas.")

Arquivos encontrados em 's3a://raw/3A_Superstore/':


+------------------------------------------+
|caminho_completo                          |
+------------------------------------------+
|s3a://raw/3A_Superstore/Order_Details.csv |
|s3a://raw/3A_Superstore/Orders.csv        |
|s3a://raw/3A_Superstore/Categories.csv    |
|s3a://raw/3A_Superstore/Customers_ENG.csv |
|s3a://raw/3A_Superstore/Customers.csv     |
|s3a://raw/3A_Superstore/Categories_ENG.csv|
|s3a://raw/3A_Superstore/Branches.csv      |
|s3a://raw/3A_Superstore/Branches_ENG.csv  |
+------------------------------------------+



[Stage 5:==============>                                          (6 + 18) / 24]


Lista de Arquivos (Python):
s3a://raw/3A_Superstore/Order_Details.csv
s3a://raw/3A_Superstore/Orders.csv
s3a://raw/3A_Superstore/Categories.csv
s3a://raw/3A_Superstore/Customers_ENG.csv
s3a://raw/3A_Superstore/Customers.csv
s3a://raw/3A_Superstore/Categories_ENG.csv
s3a://raw/3A_Superstore/Branches.csv
s3a://raw/3A_Superstore/Branches_ENG.csv
